
Tout d'abord, installer les librairies nécessaires: 


In [ ]:
!pip install --quiet transformers datasets evaluate seqeval

Charger les libraries dans l'environnement

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification

import zipfile

import evaluate
import numpy as np

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import pipeline

Uploader dans votre environment de travail l'archive zip fournie (corpusCasM2-main.zip) à l'aide de l'explorateur de fichier google colab.

Puis décompresser le fichier avec la commande suivante: 

In [ ]:
with zipfile.ZipFile("corpusCasM2-main.zip", "r") as archive:
        archive.extractall("data")

Charger le dataset corpusCasM2

In [ ]:
corpusCas = load_dataset("data/corpusCasM2-main/corpusCasM2")

Afficher un exemple du trainset:

id
: correspond à l'id du document et le numero de la phrase dans le document 

tokens
: contient le texte de la phrase pre-tokenisé

ner_tags
: contient les labels au format [BIO](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)). Ils sont affichés ici par leur ID 

In [ ]:
corpusCas["train"][0]

Il est possible de récupérer la liste des labels à partir des features du dataset: 

In [ ]:
label_list = corpusCas["train"].features["ner_tags"].feature.names
label_list

Télécharger le tokenizer du modèle choisi (ici "distilbert-base-uncased"): 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Voici un exemple d'output du tokenizer: 

In [ ]:
example = corpusCas["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

Vous pouvez constater que le texte tokenisé comprend plus de tokens que de mots initialement présents. Or les labels n'existent que pour un mot. Il faut donc réaligner les tokens et les labels en utilisant la fonction suivante: 

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_corpus = corpusCas.map(tokenize_and_align_labels, batched=True)

On obtient bien l'effet escompté: 


In [ ]:
tokenized_corpus["train"][0]

On crée ensuite un [DataCollator](https://huggingface.co/docs/transformers/main_classes/data_collator) qui va servir à créer les batchs en entrée du modèle. On utilise un DataCollator spécifique à la tache 

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Ensuite on crée une petite fonction qui va être chargée de faire l'évaluation du modèle: ici on se sert de la métrique [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval).  

In [ ]:

seqeval = evaluate.load("seqeval")
labels = [label_list[i] for i in example[f"ner_tags"]]



def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Créons à présent les dictionnaires qui permettent de lier les labels avec leurs IDs respectifs


In [ ]:
id2label = {i:label for i,label in enumerate(label_list)}
label2id = {v:k for k,v in id2label.items()}
label2id


Nous pouvons à présent instancier le modèle que nous voulons fine-tuner: 


In [ ]:

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(id2label), id2label=id2label, label2id=label2id
)

## TODO

Pour simplifier le travail, les **scopes de temporalité ne sont pas pris en compte** dans cette version du dataset. 

1. Identifier un modèle adapté à la tâche. Ici nous voulons faire de la reconnaissance d'entités nommées (classification de tokens) dans des textes en français. Ce choix devra être argumenté. Dans l'exemple, le modèle choisi n'est pas particulièrement adapté au français. 

2. Fine-tuner ce modèle sur la tâche. C'est à dire réentrainer le modèle avec les données du corpusCasM2. Vous devrez tester différentes combinaisons d'hyperparamètres afin de trouver les meilleurs. Pour cela, vous mettrez en place une stratégie d'optimisation des hyperparamètres que vous justifierez.

3. Evaluer de manière adaptée et argumentée les résultats obtenus.
